# **MIP1_Run2**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [ ]:
problem = """ You are a city planner, looking to open facilities at some locations. We have a set of customers and a set of possible locations for opening facilities. Each potential location for establishing a facility incurs a fixed annual activation cost, which must be paid if the facility is used, regardless of the service volume it handles. Furthermore, this service volume at each facility is capped at a maximum annual limit. Additionally, there are transportation costs associated with servicing each customer from each facility.
The goal is to minimize the overall costs, which include both the fixed activation costs for any opened facilities and the transportation costs for servicing customers. This must be done while making sure that each customer’s demand is met, each facility does not exceed its maximum service volume, and customers can of course only be serviced by a facility that is opened.
Please formulate this as a mathematical optimization model."""

In [ ]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [ ]:
print(response1.choices[0].message.content)

Denote:

Sets:

- I is the set of potential facility locations, indexed by i.
- J is the set of customers, indexed by j.

Parameters:

- F_i is the fixed activation cost for facility i for i ∈ I.
- C_ij is the transportation cost from facility i to customer j for i ∈ I, j ∈ J.
- D_j is the demand of customer j for j ∈ J.
- V_i is the maximum service volume of facility i for i ∈ I.

Decision variables:

- S_ij is the service volume from facility i to customer j for i ∈ I, j ∈ J.
- X_i is a binary variable that equals 1 if we activate facility i and 0 otherwise for i ∈ I.

Objective function:

Minimize total cost = Sum of (activation costs + transportation costs)
= Sum over i in I (F_i * X_i + Sum over j in J (C_ij * S_ij))

Subject to 

1. Demand constraints: For each customer j, the sum of the service volumes from all facilities must be greater than or equal to the customer's demand.  
    Sum over i in I (S_ij) ≥ D_j for all j ∈ J

2. Volume constraints: For each facility i, the sum o

### **Generate Pyomo Code**

In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [ ]:
Markdown(response2.choices[0].message.content)

Here is the Python/Pyomo implementation of your problem:

```python
from pyomo.environ import *

## You need to replace the following items with your actual data
# Sample data 
I = [1, 2, 3]    # Potential facility locations
J = [1, 2]    # Customers
F = {1: 100, 2: 120, 3: 90}    # Fixed activation cost
V = {1: 100, 2: 200, 3: 300}    # Maximum service volume
C = {(1,1): 10, (1,2): 15,
     (2,1): 20, (2,2): 25,
     (3,1): 30, (3,2): 35}    # Transportation cost
D = {1: 50, 2: 150}    # demand of customers

# Create the model
model = ConcreteModel()

# Sets definition
model.I = Set(initialize=I)
model.J = Set(initialize=J)

# Variables definition
model.X = Var(model.I, within=Binary)
model.S = Var(model.I, model.J, within=NonNegativeReals)

# Objective 
model.cost = Objective(expr = sum(F[i]*model.X[i] for i in model.I) + 
                        sum(C[i,j]*model.S[i,j] for i in model.I for j in model.J),
                        sense=minimize)

# Constraints
model.demand = ConstraintList()
for j in model.J:
    model.demand.add(sum(model.S[i,j] for i in model.I) >= D[j])

model.volume = ConstraintList()
for i in model.I:
    model.volume.add(sum(model.S[i,j] for j in model.J) <= V[i]*model.X[i])

# Pyomo provides an automated way to compile and solve these problems
solver = SolverFactory('glpk')
solver.solve(model)

# print the results
for i in model.I:
    print('Facility %s activation: ' % i, model.X[i].value)
for i in model.I:
    for j in model.J:
        print('Service volume from facility %s to customer %s: ' %(i,j), model.S[i,j].value)
```
This script creates and solves the problem using the GLPK solver. You need the GLPK solver installed on your machine to get it running.

In this script, I define two sets I and J for the facilities and customers, respectively. Then, I define the decision variables: X (for the activation of facilities) and S (for the service volume from each facility to each customer). The objective function is defined as 'cost', and I add two types of constraints to the model: 'demand' and 'volume'. Finally, I call a solver to solve the model, and print the results.

### **Run the code Generated by GPT4**

In [ ]:
from pyomo.environ import *

## You need to replace the following items with your actual data
# Sample data
I = [1, 2, 3]    # Potential facility locations
J = [1, 2]    # Customers
F = {1: 100, 2: 120, 3: 90}    # Fixed activation cost
V = {1: 100, 2: 200, 3: 300}    # Maximum service volume
C = {(1,1): 10, (1,2): 15,
     (2,1): 20, (2,2): 25,
     (3,1): 30, (3,2): 35}    # Transportation cost
D = {1: 50, 2: 150}    # demand of customers

# Create the model
model = ConcreteModel()

# Sets definition
model.I = Set(initialize=I)
model.J = Set(initialize=J)

# Variables definition
model.X = Var(model.I, within=Binary)
model.S = Var(model.I, model.J, within=NonNegativeReals)

# Objective
model.cost = Objective(expr = sum(F[i]*model.X[i] for i in model.I) +
                        sum(C[i,j]*model.S[i,j] for i in model.I for j in model.J),
                        sense=minimize)

# Constraints
model.demand = ConstraintList()
for j in model.J:
    model.demand.add(sum(model.S[i,j] for i in model.I) >= D[j])

model.volume = ConstraintList()
for i in model.I:
    model.volume.add(sum(model.S[i,j] for j in model.J) <= V[i]*model.X[i])

# Pyomo provides an automated way to compile and solve these problems
solver = SolverFactory('glpk')
solver.solve(model)

# print the results
for i in model.I:
    print('Facility %s activation: ' % i, model.X[i].value)
for i in model.I:
    for j in model.J:
        print('Service volume from facility %s to customer %s: ' %(i,j), model.S[i,j].value)

Facility 1 activation:  1.0
Facility 2 activation:  1.0
Facility 3 activation:  0.0
Service volume from facility 1 to customer 1:  0.0
Service volume from facility 1 to customer 2:  100.0
Service volume from facility 2 to customer 1:  50.0
Service volume from facility 2 to customer 2:  50.0
Service volume from facility 3 to customer 1:  0.0
Service volume from facility 3 to customer 2:  0.0


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [3]:
from pyomo.environ import *

## data inputted by human
I = [1, 2, 3]    # Potential facility locations
J = [1,2,3,4,5]    # Customers
F = {1: 1000, 2: 1000, 3: 1000}    # Fixed activation cost
V = {1: 500, 2: 500, 3: 500}    # Maximum service volume
C = {
    (1, 1): 4,  (2, 1): 6,  (3, 1): 9,
    (1, 2): 5,  (2, 2): 4,  (3, 2): 7,
    (1, 3): 6,  (2, 3): 3,  (3, 3): 4,
    (1,4): 8,  (2,4): 5,  (3,4): 3,
    (1,5): 10, (2,5): 8,  (3,5): 4,
}    # Transportation cost
D = {1: 80, 2: 270, 3: 250, 4: 160, 5: 180}    # demand of customers
#end
# Create the model
model = ConcreteModel()

# Sets definition
model.I = Set(initialize=I)
model.J = Set(initialize=J)

# Variables definition
model.X = Var(model.I, within=Binary)
model.S = Var(model.I, model.J, within=NonNegativeReals)

# Objective
model.cost = Objective(expr = sum(F[i]*model.X[i] for i in model.I) +
                        sum(C[i,j]*model.S[i,j] for i in model.I for j in model.J),
                        sense=minimize)

# Constraints
model.demand = ConstraintList()
for j in model.J:
    model.demand.add(sum(model.S[i,j] for i in model.I) >= D[j])

model.volume = ConstraintList()
for i in model.I:
    model.volume.add(sum(model.S[i,j] for j in model.J) <= V[i]*model.X[i])

model.facility_activation = ConstraintList()
for i in I:
  for j in J:
    model.facility_activation.add(model.S[i,j] <= V[i]*model.X[i])

# Pyomo provides an automated way to compile and solve these problems
solver = SolverFactory('glpk')
solver.solve(model)

# print the results
print(model.cost())
for i in model.I:
    print('Facility %s activation: ' % i, model.X[i].value)
for i in model.I:
    for j in model.J:
        print('Service volume from facility %s to customer %s: ' %(i,j), model.S[i,j].value)

5610.0
Facility 1 activation:  0.0
Facility 2 activation:  1.0
Facility 3 activation:  1.0
Service volume from facility 1 to customer 1:  0.0
Service volume from facility 1 to customer 2:  0.0
Service volume from facility 1 to customer 3:  0.0
Service volume from facility 1 to customer 4:  0.0
Service volume from facility 1 to customer 5:  0.0
Service volume from facility 2 to customer 1:  80.0
Service volume from facility 2 to customer 2:  270.0
Service volume from facility 2 to customer 3:  150.0
Service volume from facility 2 to customer 4:  0.0
Service volume from facility 2 to customer 5:  0.0
Service volume from facility 3 to customer 1:  0.0
Service volume from facility 3 to customer 2:  0.0
Service volume from facility 3 to customer 3:  100.0
Service volume from facility 3 to customer 4:  160.0
Service volume from facility 3 to customer 5:  180.0
